<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/bike_sharing_count_predict(REGRESSION).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2022-11-21T06:17:23.093280Z","iopub.execute_input":"2022-11-21T06:17:23.095423Z","iopub.status.idle":"2022-11-21T06:17:26.204835Z"}}
# 시험환경 세팅 (코드 변경 x)
#Bike-sharing-demand
library(dplyr)
library(caret)
library(ModelMetrics)
library(lubridate)
library(randomForest)
train<-read.csv('../input/bike-sharing-demand/train.csv')
test<-read.csv('../input/bike-sharing-demand/test.csv')

train$datetime<-as.POSIXct(train$datetime, format="%Y-%m-%d %H:%M:%S")

train<-train%>%mutate(hour=hour(datetime))
train$hour <- as.factor(train$hour)
train$season <- as.factor(train$season)
train$holiday <- as.factor(train$holiday)
train$workingday <- as.factor(train$workingday)
train$weather <- as.factor(train$weather)

test$datetime<-as.POSIXct(test$datetime, format="%Y-%m-%d %H:%M:%S")
test<-test%>%mutate(hour=hour(datetime))
test$hour <- as.factor(test$hour)
test$season <- as.factor(test$season)
test$holiday <- as.factor(test$holiday)
test$workingday <- as.factor(test$workingday)
test$weather <- as.factor(test$weather)



#colSums(is.na(train)) #결측치 없음
#cor(train$registered,train$count)  #registered 와 count 상관관계 0.97

train<-train[,-1]
#test<-test[,-1]
idx<-sample(1:nrow(train),0.7*nrow(train))
x_train<- train[idx,]
x_test<- train[-idx,]



# count종속변수
m_rf<-randomForest(count~.,x_train,ntree=100)
pred_rf<-predict(m_rf,x_test[,-11])
rmse(pred_rf,x_test[,11]) ## RMSE:12.7652
R2(pred_rf,x_test[,11])   ## R2score:0.996


## registered 종속변수
mr_rf<-randomForest(registered~.,x_train,ntree=100)
predr_rf<-predict(mr_rf,x_test[,-10])
rmse(predr_rf,x_test[,10]) ## RMSE:13.6788
R2(predr_rf,x_test[,10])   ## R2score:0.9992


## casual 종속변수
mc_rf<-randomForest(casual~.,x_train,ntree=100)
predc_rf<-predict(mc_rf,x_test[,-9])
rmse(predc_rf,x_test[,9]) ## RMSE:11.692
R2(predc_rf,x_test[,9])   ## R2score:0.954


#test셋 모델링 및 예측
m2_rf<-randomForest(count~.,train[,-c(9,10)],ntree=100)
pred2_rf<-predict(m2_rf,test[,-c(1)])

#결과 제출
result<-cbind(as.character(test[,1]),as.character(pred2_rf))
colnames(result)<-c("datetime","count")
head(result)
write.csv(result,"0099.csv",row.names=F)
zz<-read.csv("0099.csv")
head(zz)

